In [4]:
import torch


# Setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:,  ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3070
Memory Usage:
Allocated: 2.2 GB
Cached:    2.2 GB


In [3]:
import time


device = torch.device('cuda')
x = torch.rand((10000, 10000), dtype=torch.float64)
y = torch.rand((10000, 10000), dtype=torch.float64)
x = x.to(device)
y = y.to(device)

start = time.time()
res = x * y
print(time.time(), - start)

0.009374856948852539


In [15]:
import os
import glob
import shutil

# glob.glob('/media/des/T5 EVO 3/data/route_59_14032024/drive_1/pcd/*.pcd')
files = glob.glob('/Volumes/proj-4320_tram_digital_twin-1128.4.913/data/route_59_14032024/drive_1/pcd/*.pcd')
dst_dir = './data/route_59_14032024_down/'

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

os.mkdir(dst_dir)

# last = None
# for i in range(0, len(files), 5):
for i in range(30150, 30350):
    f = files[i]
    ts = int(f.split('/')[-1].split('.')[0])
    # if last is not None and ts < last + 1e9:
    #     continue
    os.symlink(f, os.path.join(dst_dir, f.split('/')[-1]))
    last = ts

In [29]:
import numpy as np
import torch
from scipy.spatial.transform import Rotation as R
import time


def rpyd2rotm(rpyd):
    rotation = R.from_euler('xyz', rpyd, True)
    rotation_matrix = rotation.as_matrix()
    return rotation_matrix

def invert_transformation(transformation):
    """
    Compute the inverse of a transformation matrix without inverting it.

    Args:
    transformation: A 4 by 4 matrix representing the transformation.

    Returns:
    transformation_inv: A 4 by 4 matrix representing the inverse transformation.
    """
    rotm = transformation[:3, :3]
    translation = transformation[:3, 3]

    transformation_inv = np.eye(4)
    transformation_inv[:3, :3] = rotm.T
    transformation_inv[:3, 3] = -rotm.T @ translation
    return transformation_inv


r = rpyd2rotm(np.array([30, 30, 30]))
t = np.array([10, 10, -5]).T
tr = np.eye(4)
tr[:3, :3] = r
tr[:3, 3] = t

print(tr)

tr_torch = torch.from_numpy(tr)
tr_torch = tr_torch.to(dtype=torch.float64, device='cpu')
print(tr_torch)

tr_torch = tr_torch.to(dtype=torch.float32, device='mps')
print(tr_torch)

start = time.time()
print(np.linalg.inv(tr))
end = time.time()
print(end - start)

start = time.time()
print(invert_transformation(tr))
end = time.time()
print(end - start)

[[ 0.75       -0.21650635  0.625      10.        ]
 [ 0.4330127   0.875      -0.21650635 10.        ]
 [-0.5         0.4330127   0.75       -5.        ]
 [ 0.          0.          0.          1.        ]]
tensor([[ 0.7500, -0.2165,  0.6250, 10.0000],
        [ 0.4330,  0.8750, -0.2165, 10.0000],
        [-0.5000,  0.4330,  0.7500, -5.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)
tensor([[ 0.7500, -0.2165,  0.6250, 10.0000],
        [ 0.4330,  0.8750, -0.2165, 10.0000],
        [-0.5000,  0.4330,  0.7500, -5.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], device='mps:0')
[[  0.75         0.4330127   -0.5        -14.33012702]
 [ -0.21650635   0.875        0.4330127   -4.41987298]
 [  0.625       -0.21650635   0.75        -0.33493649]
 [  0.           0.           0.           1.        ]]
0.0008170604705810547
[[  0.75         0.4330127   -0.5        -14.33012702]
 [ -0.21650635   0.875        0.4330127   -4.41987298]
 [  0.625       -0.21650635   0.75 